In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import math
import datetime as dt
from scipy.stats import norm

### Analytical Option Formulae

The Black-Scholes formula for a call option is given by

\begin{equation}
\begin{split}
C(S,K,r,\sigma,T) &= S_0 \Phi(d_1) - K e^{-rT} \Phi(d_2)\\
            d_1 &= \frac{\log \frac{S_0}{K} +
            \left(r+\frac{\sigma^2}{2}\right)T}{\sigma\sqrt{T}}, \hspace{2cm} d_2 = d_1 - \sigma\sqrt{T}\\
\end{split}            
\end{equation}

The formula can be implemented in Python as follows:

In [2]:
#### Value Vanilla call/put
    
    
# Black-Scholes model
def Vanilla_BS(S,K,sigma,r,T,option_type = "call"):
    
    # Function calculates option prices based on Black-scholes model.

    # Inputs:
    # S0: underlying asset price at time 0; can be an array of prices
    # K: strike price; can be an array of prices
    # r: interest rate, annualized
    # T: time to expiration (also the T-t in our equations), in number of years
    # sigma: implied volatility of the option
    # call: default True. True if pricing call options; otherwise False

    # Outputs: Option Prices.

    d1 = (np.log(S/K) + (r + sigma**2*0.5)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        option_price = S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == "put":
        option_price = K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return option_price


# Bachelier model
def Vanilla_Bachelier(S,K,sigma,r,T,option_type = "call"):
    x_star = (K - S) / (sigma * np.sqrt(T))
    
    if option_type == "call":
        option_price = np.exp(-r*T) * ((S - K) * norm.cdf(-x_star) + sigma * np.sqrt(T) *norm.pdf(-x_star))
    elif option_type == "put":
        option_price = np.exp(-r*T) * ((K - S) * norm.cdf(x_star) + sigma * np.sqrt(T) *norm.pdf(x_star))
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return option_price


# Black-76 model
def Vanilla_Black76(F,K,sigma,r,T,option_type = "call"):
    d1 = (np.log(F/K) + 0.5 * (sigma**2) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        option_price = np.exp(-r*T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == "put":
        option_price = np.exp(-r*T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("Option type must be 'call' or 'put'")
        
    return option_price


# Displaced-diffusion model
def Vanilla_Displaced_Diffusion(F,K,sigma,r,T,beta,option_type = "call"):
    F0 = F / beta
    K0 = K + (1-beta)/beta * F
    sigma0 = sigma * beta
    
    if option_type == "call":
        option_price = Vanilla_Black76(F0,K0,sigma0,r,T,option_type = "call")
    elif option_type == "put":
        option_price = Vanilla_Black76(F0,K0,sigma0,r,T,option_type = "put")
    
    return option_price

In [3]:
### Value Digital cash-or-nothing call/put


# use Black-Scholes model
def Digital_cash_or_nothing_BS(S,K,sigma,r,T,Q,option_type = "call"):
    d1 = (np.log(S/K) + (r + sigma**2*0.5)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        cash_payoff = Q * np.exp(-r*T)*norm.cdf(d2)
    elif option_type == "put":
        cash_payoff = Q * np.exp(-r*T)*norm.cdf(-d2)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    return cash_payoff


# use Bachelier model
def Digital_cash_or_nothing_Bachelier(S,K,sigma,r,T,Q,option_type = "call"):
    x_star = (K - S) / (sigma * np.sqrt(T))
    
    if option_type == "call":
        cash_payoff = Q * np.exp(-r*T) * norm.cdf(- x_star)
    elif option_type == "put":
        cash_payoff = Q * np.exp(-r*T) * norm.cdf(x_star)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return cash_payoff


# use Black-76 model
def Digital_cash_or_nothing_Black76(F,K,sigma,r,T,Q,option_type = "call"):
    x = (np.log(K/F) + 0.5**(sigma**2 * T)) / (sigma * np.sqrt(T))
    
    if option_type == "call":
        cash_payoff = Q * np.exp(-r*T) * norm.cdf(-x)
    elif option_type == "put":
        cash_payoff = Q * np.exp(-r*T) * norm.cdf(x)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
        
    return cash_payoff


# use Displaced-diffusion model
def Digital_cash_or_nothing_Displaced_Diffusion(F,K,sigma,r,T,Q,beta,option_type = "call"):
    F0 = F / beta
    K0 = K + (1-beta)/beta * F
    sigma0 = sigma * beta
    
    if option_type == "call":
        cash_payoff = Digital_cash_or_nothing_Black76(F0,K0,sigma0,r,T,Q,option_type = "call")
    elif option_type == "put":
        cash_payoff = Digital_cash_or_nothing_Black76(F0,K0,sigma0,r,T,Q,option_type = "put")
    
    return cash_payoff

In [4]:
### Value Digital asset-or-nothing call/put


# use Black-Scholes model
def Digital_asset_or_nothing_BS(S,K,sigma,r,T,option_type = "call"):
    d1 = (np.log(S/K) + (r + sigma**2*0.5)*T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        asset_payoff = S*norm.cdf(d1)
    elif option_type == "put":
        asset_payoff = S*norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    return asset_payoff  

# use Bachelier model
def Digital_asset_or_nothing_Bachelier(S,K,sigma,r,T,option_type = "call"):
    x_star = (K - S) / (sigma * np.sqrt(T))
    
    if option_type == "call":
        asset_payoff = np.exp(-r*T) * (S* norm.cdf(- x_star) + sigma * np.sqrt(T) *norm.pdf(- x_star))
    elif option_type == "put":
        asset_payoff = np.exp(-r*T) * (S* norm.cdf(x_star) - sigma * np.sqrt(T) *norm.pdf(x_star))
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    
    return asset_payoff


# use Black-76 model
def Digital_asset_or_nothing_Black76(F,K,sigma,r,T,option_type = "call"):
    x = (np.log(K/F) + 0.5**(sigma**2 * T)) / (sigma * np.sqrt(T))
    
    if option_type == "call":
        asset_payoff = np.exp(-r*T) * F * norm.cdf(sigma * np.sqrt(T) - x)
    elif option_type == "put":
        asset_payoff = np.exp(-r*T) * F * norm.cdf(x - sigma * np.sqrt(T))
    else:
        raise ValueError("Option type must be 'call' or 'put'")
        
    return asset_payoff


# use Displaced-diffusion model
def Digital_asset_or_nothing_Displaced_Diffusion(F,K,sigma,r,T,beta,option_type = "call"):
    F0 = F / beta
    K0 = K + (1-beta)/beta * F
    sigma0 = sigma * beta
    
    if option_type == "call":
        asset_payoff = Digital_asset_or_nothing_Black76(F0,K0,sigma0,r,T,option_type = "call")
    elif option_type == "put":
        asset_payoff = Digital_asset_or_nothing_Black76(F0,K0,sigma0,r,T,option_type = "put")
    
    return asset_payoff

In [14]:
S0 = 100
K = 150
r = 0.1
T = 20
#F = S * np.exp(r*T)
F=100
sigma = 0.2

beta = 0.5

Q = 1

In [11]:
print(f'Vanilla_BS call is {Vanilla_BS(S0,K,sigma,r,T,option_type = "call")}')
print(f'Vanilla_BS put is {Vanilla_BS(S0,K,sigma,r,T,option_type = "put")}')
print(f'Vanilla_Bachelier call is {Vanilla_Bachelier(S,K,sigma,r,T,option_type = "call")}')
print(f'Vanilla_Bachelier put is {Vanilla_Bachelier(S,K,sigma,r,T,option_type = "put")}')
print(f'Vanilla_Black76 call is {Vanilla_Black76(F,K,sigma,r,T,option_type = "call")}')
print(f'Vanilla_Black76 put is {Vanilla_Black76(F,K,sigma,r,T,option_type = "put")}')
print(f'Vanilla_Displaced_Diffusion call is {Vanilla_Displaced_Diffusion(F,K,sigma,r,T,beta,option_type = "call")}')
print(f'Vanilla_Displaced_Diffusion put is {Vanilla_Displaced_Diffusion(F,K,sigma,r,T,beta,option_type = "put")}')


print("=======================")
print(f'Digital_cash_or_nothing_BS call is {Digital_cash_or_nothing_BS(S,K,sigma,r,T,Q,option_type = "call")}')
print(f'Digital_cash_or_nothing_BS put is {Digital_cash_or_nothing_BS(S,K,sigma,r,T,Q,option_type = "put")}')
print(f'Digital_cash_or_nothing_Bachelier call is {Digital_cash_or_nothing_Bachelier(S,K,sigma,r,T,Q,option_type = "call")}')
print(f'Digital_cash_or_nothing_Bachelier put is {Digital_cash_or_nothing_Bachelier(S,K,sigma,r,T,Q,option_type = "put")}')
print(f'Digital_cash_or_nothing_Black76 call is {Digital_cash_or_nothing_Black76(F,K,sigma,r,T,Q,option_type = "call")}')
print(f'Digital_cash_or_nothing_Black76 put is {Digital_cash_or_nothing_Black76(F,K,sigma,r,T,Q,option_type = "put")}')
print(f'Digital_cash_or_nothing_Displaced_Diffusion call is {Digital_cash_or_nothing_Displaced_Diffusion(F,K,sigma,r,T,Q,beta,option_type = "call")}')
print(f'Digital_cash_or_nothing_Displaced_Diffusion put is {Digital_cash_or_nothing_Displaced_Diffusion(F,K,sigma,r,T,Q,beta,option_type = "put")}')


print("=======================")
print(f'Digital_asset_or_nothing_BS call is {Digital_asset_or_nothing_BS(S,K,sigma,r,T,option_type = "call")}')
print(f'Digital_asset_or_nothing_BS put is {Digital_asset_or_nothing_BS(S,K,sigma,r,T,option_type = "put")}')
print(f'Digital_asset_or_nothing_Bachelier call is {Digital_asset_or_nothing_Bachelier(S,K,sigma,r,T,option_type = "call")}')
print(f'Digital_asset_or_nothing_Bachelier put is {Digital_asset_or_nothing_Bachelier(S,K,sigma,r,T,option_type = "put")}')
print(f'Digital_asset_or_nothing_Black76 call is {Digital_asset_or_nothing_Black76(F,K,sigma,r,T,option_type = "call")}')
print(f'Digital_asset_or_nothing_Black76 put is {Digital_asset_or_nothing_Black76(F,K,sigma,r,T,option_type = "put")}')
print(f'Digital_asset_or_nothing_Displaced_Diffusion call is {Digital_asset_or_nothing_Displaced_Diffusion(F,K,sigma,r,T,beta,option_type = "call")}')
print(f'Digital_asset_or_nothing_Displaced_Diffusion put is {Digital_asset_or_nothing_Displaced_Diffusion(F,K,sigma,r,T,beta,option_type = "put")}')



Vanilla_BS call is 80.25650471103204
Vanilla_BS put is 0.5567971965239404
Vanilla_Bachelier call is 0.0
Vanilla_Bachelier put is 6.766764161830635
Vanilla_Black76 call is 80.25650471103205
Vanilla_Black76 put is 0.5567971965239402
Vanilla_Displaced_Diffusion call is 84.05226508136474
Vanilla_Displaced_Diffusion put is 4.352557566856618
Digital_cash_or_nothing_BS call is 0.12303988708734406
Digital_cash_or_nothing_BS put is 0.012295396149268636
Digital_cash_or_nothing_Bachelier call is 0.0
Digital_cash_or_nothing_Bachelier put is 0.1353352832366127
Digital_cash_or_nothing_Black76 call is 0.1181453028904789
Digital_cash_or_nothing_Black76 put is 0.017189980346133803
Digital_cash_or_nothing_Displaced_Diffusion call is 0.028282657604032407
Digital_cash_or_nothing_Displaced_Diffusion put is 0.10705262563258029
Digital_asset_or_nothing_BS call is 98.71248777413365
Digital_asset_or_nothing_BS put is 1.287512225866355
Digital_asset_or_nothing_Bachelier call is 0.0
Digital_asset_or_nothing_Bach

In [15]:
Vanilla_Black76(F,K,sigma,r,T,option_type = "call")

3.000202501035807